In [ ]:
!pip install chromadb
!git clone https://huggingface.co/datasets/NahedAbdelgaber/evaluating-student-writing
!git clone  https://huggingface.co/datasets/transformersbook/emotion-train-split
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import chromadb as chdb
from chromadb.utils import embedding_functions
chroma = chdb.PersistentClient(path="chroma_db_files")
collection_name = "Rag_using_HuggingFace_Embeddings"
collection = chroma.create_collection(name=collection_name)

In [ ]:
def split_text_into_chunks(text, chunk_size=1000, chunk_overlap=20):
  chunks = []
  start = 0
  while start < len(text):
    end = start + chunk_size
    chunks.append(text[start:end])
    start = end
  return chunks

In [ ]:
import os
directory = '/content/evaluating-student-writing/training-corpus.txt'
with open(directory, 'r') as file:
    text = file.read()

chunked_data = split_text_into_chunks(text)
print(len(chunked_data))

36866


In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#embeddings = embed_model.get_text_embedding(chunked_data)
embeddings = embed_model.get_text_embedding_batch(chunked_data)
print(len(embeddings))
print(embeddings[:5])

36866
[[0.006823241710662842, -0.006791205611079931, -0.013492577709257603, -0.00760646490380168, 0.02009204588830471, -0.08858568966388702, 0.05094531178474426, 0.021793127059936523, 0.020695772022008896, -0.058135949075222015, 0.028172830119729042, -0.017255892977118492, -0.0361514575779438, 0.0335712768137455, -0.04950619488954544, 0.10604401677846909, 0.0531294122338295, 0.03518263250589371, 0.026928367093205452, 0.01947377808392048, 0.0021104468032717705, 0.03245578706264496, 0.01631877012550831, 0.06385162472724915, 0.053857818245887756, 0.008390662260353565, 0.05675055831670761, 0.0675344243645668, -0.005846999119967222, -0.048276081681251526, 0.061969611793756485, -0.019866829738020897, 0.0055213430896401405, -0.017491426318883896, 0.00020523101557046175, -0.06081628426909447, 0.012447558343410492, 0.031239373609423637, 0.0010663969442248344, -0.07598745822906494, -0.03903066739439964, -0.003802708350121975, -0.04349789395928383, -0.033979155123233795, -0.046212632209062576, 0.

In [ ]:
ids = [f"chunk-{i}" for i in range(len(chunked_data))]
collection.upsert(
    documents=chunked_data,
    metadatas=None,
    embeddings=embeddings,
    ids=ids  # Use the generated IDs
)

In [ ]:
from multiprocessing import context
def query_collection(query_text, n_results=2, top_k=3):
    query_embedding = embed_model.get_text_embedding(query_text)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )
    return results

In [ ]:
query_collection(" Even though the planet is very similar to Earth there are challenges to get accurate data on the planet because of the many spacecrafts that were unable to withstand the harshness of the planet. As technology advanced, the author claims that more and more missions to Venus occured, and even around the time of World War II there was a spacecraft that survived in Venus conditions for about three weeks." , n_results=1)

{'ids': [['chunk-25277']],
 'distances': [[0.32882219552993774]],
 'metadatas': [[None]],
 'embeddings': None,
 'documents': [[' with oceans and could have supported various forms of life, just like Earth." In addition, it also says," Some simplified electronics made of silicon carbide have been tested in a chamber simulating the chaos of Venus\'s surface and have lasted for three weeks in such conditions." There are a lot of astronauts in NASA that are willing to take on the challenge to Venus and NASA has already been simulatiing prototypes to withstand the conditions similar to Venus. The planet is not just fascinating. it\'s also really dangerous for astronauts to travel there.\n\nExploring Venus is really dangerous that no one will be able to survive with such harsh conditions the astronauts have to face. In the text it says, " Even more challenging are the clouds of highly corrosive sulfuric acid in Venus\'s atmosphere. On the planet\'s surface, temperatures average over 800 degr